## Link to datasets


In [0]:
import io
import requests
import pandas as pd
import pyspark.sql.functions as f
from delta.tables import DeltaTable
from pyspark.sql.types import (
    StructType, StructField,
    IntegerType, StringType, DoubleType, TimestampType
)
import json

In [0]:
# spark.sql("DROP TABLE IF EXISTS airport_delays.raw.apt_dly")

In [0]:
contracts_path = "/Volumes/airport_delays/raw/vol/contracts"

In [0]:
def load_data_from_url_as_spark_df(url, **params):
    """Load CSV (incl. .bz2) from URL into Spark DataFrame."""
    response = requests.get(url)
    response.raise_for_status()
    pdf = pd.read_csv(io.BytesIO(response.content), **params)
    return spark.createDataFrame(pdf)

def update_schema_with_metadata_fields(schema):
    return (
        schema
        .add("processing_datetime", TimestampType(), True)
        .add("source_year", IntegerType(), True)
        .add("dataset_group", StringType(), True)
    )

def add_metadata_columns(df, year: int, dataset_group: str):
    return (
        df
        .withColumn("processing_datetime", f.current_timestamp())
        .withColumn("source_year", f.lit(year))
        .withColumn("dataset_group", f.lit(dataset_group))
    )


The Airport Arrival ATFM Delay provides an indication of ATFM delays on the ground due to constraints at airports. <br>

In Europe, when traffic demand is anticipated to exceed the available capacity in en route centres or at airports, Air Traffic Control (ATC ) units may request the local Flow Management Position (FMP ) to instigate an Air Traffic Flow Management (ATFM ) measure, or regulation . Aircraft expected to arrive during a period of congestion are given ATFM delay at their departure airport, under the authority of the Network Manager, in order to regulate the flow of traffic into the constrained downstream en route sector or airport, thus ensuring safety. <br>

The resulting ATFM delays are calculated as the difference between the estimated take-off time calculated from the filed flight plan including updates and the calculated take-off time allocated by the central unit of ATFM. The reason for the regulation is indicated by the responsible FMP. The delay is attributed to the most constraining ATC unit. <br>

The calculation of Airport arrival ATFM delay is based on a well established and commonly accepted algorithm and has been in use as a commonly agreed proxy for airport capacity shortfalls since 2009. <br>

| Column name         | Data source      | Label                  | Reason Group     | Column description                                                    | Example  |
|---------------------|------------------|------------------------|-----------------|---------------------------------------------------------------------|----------|
| YEAR                | Network Manager  | YEAR                   |                 | Reference year                                                      | 2015     |
| MONTH_NUM           | Network Manager  | MONTH                  |                 | Month (numeric)                                                    | 2        |
| MONTH_MON           | Network Manager  | MONTH_MON              |                 | Month (3-letter code)                                              | FEB      |
| FLT_DATE            | Network Manager  | FLT_DATE               |                 | Date of the flight                                                 | 11/02/2015 (*) |
| APT_ICAO            | Network Manager  | APT_ICAO               |                 | ICAO 4-letter airport designator                                  | LSGG     |
| APT_NAME            | PRU              | APT_NAME               |                 | Airport name                                                      | Geneva   |
| STATE_NAME          | PRU              | STATE_NAME             |                 | Name of the country in which the airport is located              | Switzerland |
| FLT_ARR_1           | Network Manager  | IFR Arrivals           |                 | Number of arrivals (based on activated flight plans submitted to NM) | 221      |
| DLY_APT_ARR_1       | Network Manager  | Airport ATFM arrival delay |              | Minutes of airport arrival ATFM delay                             | 1312     |
| DLY_APT_ARR_A_1     | Network Manager  | A - Accident/Incident - AD | AD Disruptions | Minutes of airport arrival ATFM delay with delay code A - Accident/Incident | 0        |
| DLY_APT_ARR_C_1     | Network Manager  | C - ATC Capacity - AD  | AD Capacity (ATC) | Minutes of airport arrival ATFM delay with delay code C - ATC Capacity | 0        |
| DLY_APT_ARR_D_1     | Network Manager  | D - De-icing - AD      | AD Weather      | Minutes of airport arrival ATFM delay with delay code D - De-icing | 0        |
| DLY_APT_ARR_E_1     | Network Manager  | E - Equipment (non-ATC) - AD | AD Disruptions | Minutes of airport arrival ATFM delay with delay code E - Equipment (non-ATC) | 0        |
| DLY_APT_ARR_G_1     | Network Manager  | G - Aerodrome Capacity - AD | AD Capacity   | Minutes of airport arrival ATFM delay with delay code G - Aerodrome Capacity | 0        |
| DLY_APT_ARR_I_1     | Network Manager  | I - Industrial Action (ATC) - AD | AD Disruptions (ATC) | Minutes of airport arrival ATFM delay with delay code I - Industrial Action (ATC) | 0        |
| DLY_APT_ARR_M_1     | Network Manager  | M - Airspace Management - AD | AD Capacity   | Minutes of airport arrival ATFM delay with delay code M - Airspace Management | 0        |
| DLY_APT_ARR_N_1     | Network Manager  | N - Industrial Action (non-ATC) - AD | AD Disruptions | Minutes of airport arrival ATFM delay with delay code N - Industrial Action (non-ATC) | 0        |
| DLY_APT_ARR_O_1     | Network Manager  | O - Other - AD         | AD Disruptions | Minutes of airport arrival ATFM delay with delay code O - Other   | 0        |
| DLY_APT_ARR_P_1     | Network Manager  | P - Special Event - AD | AD Events      | Minutes of airport arrival ATFM delay with delay code P - Special Event | 0        |
| DLY_APT_ARR_R_1     | Network Manager  | R - ATC Routeing - AD  | AD Capacity    | Minutes of airport arrival ATFM delay with delay code R - ATC Routeing | 0        |
| DLY_APT_ARR_S_1     | Network Manager  | S - ATC Staffing - AD  | AD Staffing (ATC) | Minutes of airport arrival ATFM delay with delay code S - ATC Staffing | 1312     |
| DLY_APT_ARR_T_1     | Network Manager  | T - Equipment (ATC) - AD | AD Disruptions (ATC) | Minutes of airport arrival ATFM delay with delay code T - Equipment (ATC) | 0        |
| DLY_APT_ARR_V_1     | Network Manager  | V - Environmental Issues - AD | AD Capacity | Minutes of airport arrival ATFM delay with delay code V - Environmental Issues | 0        |
| DLY_APT_ARR_W_1     | Network Manager  | W - Weather - AD       | AD Weather    | Minutes of airport arrival ATFM delay with delay code W - Weather | 0        |
| DLY_APT_ARR_NA_1    | Network Manager  | NA - Not specified - AD | AD Disruptions | Minutes of airport arrival ATFM delay with delay code NA - Not specified | 0        |
| FLT_ARR_1_DLY       | Network Manager  | FLT_ARR_1_DLY          |                 | Number of airport ATFM arrival delayed arrivals                 | 0        |
| FLT_ARR_1_DLY_15    | Network Manager  | FLT_ARR_1_DLY_15       |                 | Number of airport ATFM arrival delayed arrivals (>15 min.)      | 0        |


Data is aggregated from 2014 to 2024 year in table **raw.apt_dly** <br>
Link to description -> https://ansperformance.eu/reference/dataset/airport-arrival-atfm-delay/

In [0]:
with open(contracts_path + "/apt_dly_schema.json", "r") as fin:
    apt_dly_schema = StructType.fromJson(json.loads(fin.read()))

raw_apt_dly_schema = update_schema_with_metadata_fields(apt_dly_schema)

In [0]:
print(raw_apt_dly_schema)

In [0]:
apt_dly_delta = DeltaTable.createIfNotExists(spark) \
    .tableName("airport_delays.raw.apt_dly") \
    .addColumns(raw_apt_dly_schema) \
    .execute()


In [0]:
base_url = "https://www.eurocontrol.int/performance/data/download/csv/"

In [0]:
def normalize_apt_dly_schema(df):
    df = df.withColumn("YEAR", f.col("YEAR").cast("int")) \
           .withColumn("MONTH_NUM", f.col("MONTH_NUM").cast("int")) \
           .withColumn("FLT_ARR_1", f.col("FLT_ARR_1").cast("int")) \
           .withColumn("FLT_ARR_1_DLY", f.col("FLT_ARR_1_DLY").cast("int")) \
           .withColumn("FLT_ARR_1_DLY_15", f.col("FLT_ARR_1_DLY_15").cast("int"))
    
    delay_columns = [
        "DLY_APT_ARR_1",
        "DLY_APT_ARR_A_1",
        "DLY_APT_ARR_C_1",
        "DLY_APT_ARR_D_1",
        "DLY_APT_ARR_E_1",
        "DLY_APT_ARR_G_1",
        "DLY_APT_ARR_I_1",
        "DLY_APT_ARR_M_1",
        "DLY_APT_ARR_N_1",
        "DLY_APT_ARR_O_1",
        "DLY_APT_ARR_P_1",
        "DLY_APT_ARR_R_1",
        "DLY_APT_ARR_S_1",
        "DLY_APT_ARR_T_1",
        "DLY_APT_ARR_V_1",
        "DLY_APT_ARR_W_1",
        "DLY_APT_ARR_NA_1"
    ]
    
    for col_name in delay_columns:
        if col_name in df.columns:
            df = df.withColumn(col_name, f.col(col_name).cast("double"))
    
    return df


for year in range(2014, 2025):
    url = f"{base_url}apt_dly_{year}.csv.bz2"
    print(f"Loading apt_dly {year}")

    df = load_data_from_url_as_spark_df(
        url,
        sep=",",
        quotechar='"',
        compression="bz2"
    )
    df = normalize_apt_dly_schema(df)

    (
        df
        .transform(lambda x: add_metadata_columns(x, year, "apt_dly"))
        .writeTo("airport_delays.raw.apt_dly")
        .option("mergeSchema", "true")
        .append()
    )



In [0]:
delta_table = DeltaTable.forName(spark, "airport_delays.raw.apt_dly")
df = delta_table.toDF()
df.printSchema()
df.show(5, truncate=False)
print(f"Total rows: {df.count()}")

The ATC pre-departure delay is a proxy for ATC induced delays at the departure stand to avoid queuing at the departure runway <br>

The data is based on IATA delay code 89 [see (IATA 2012) ] which, besides delays caused by local ATC constraints, also includes delays due to late push-back approval and some other reasons which may introduce a certain level of bias. Work is in progress to allow for a better identification of the different causal factors. <br>

| Column name    | Data Source       | Label                    | Description                                    | Example      |
|----------------|-------------------|--------------------------|------------------------------------------------|--------------|
| YEAR           | Network Manager   | YEAR                     | Reference year                                 | 2014         |
| MONTH_NUM      | Network Manager   | MONTH_NUM                | Month (numeric)                                | 01           |
| MONTH_MON      | Network Manager   | MONTH_MON                | Month (3-letter code)                          | JAN          |
| FLT_DATE       | Network Manager   | FLT_DATE                 | Date of flight                                 | 01-Jan-2014  |
| APT_ICAO       | Network Manager   | APT_ICAO                 | ICAO 4-letter airport designator              | EDDK         |
| APT_NAME       | PRU               | APT_NAME                 | Airport name                                   | Cologne-Bonn |
| STATE_NAME     | PRU               | STATE_NAME               | Name of the State in which the airport is located | Germany      |
| FLT_DEP_1      | Network Manager   | IFR departures - (NM)    | Number of IFR departures from Network Manager | 66           |
| FLT_DEP_IFR_2  | Airport Operator  | IFR departures - (APT)   | Number of IFR departures submitted by airport operators | 66           |
| DLY_ATC_PRE_2  | Airport Operator  | ATC pre-departure delay (APT) | Minutes of delay reported as IATA Code 89 | 0            |
| FLT_DEP_3      | CODA/ Airlines    | IFR departures - (AL)    | Number of IFR departures submitted by airlines | 52           |
| DLY_ATC_PRE_3  | CODA/ Airlines    | ATC pre-departure delay (AL) | Minutes of delay reported as IATA Code 89 | 2            |


Data is aggregated from 2016 to 2024 years in table **raw.atc_pre_dep** <br>
Data link -> https://ansperformance.eu/reference/dataset/atc-pre-departure-delay/


In [0]:
# spark.sql("DROP TABLE IF EXISTS airport_delays.raw.atc_pre_dep")

In [0]:
with open(contracts_path + "/atc_pre_dep_schema.json", "r") as fin:
    atc_pre_dep_schema = StructType.fromJson(json.loads(fin.read()))

raw_atc_pre_dep_schema = update_schema_with_metadata_fields(atc_pre_dep_schema)

In [0]:
atc_pre_dep_delta = DeltaTable.createIfNotExists(spark) \
    .tableName("airport_delays.raw.atc_pre_dep") \
    .addColumns(raw_atc_pre_dep_schema) \
    .execute()


In [0]:
base_url = "https://www.eurocontrol.int/performance/data/download/csv/"

In [0]:
def normalize_atc_pre_dep_schema(df):
    return (
        df
        .withColumn("YEAR", f.col("YEAR").cast("int"))
        .withColumn("MONTH_NUM", f.col("MONTH_NUM").cast("int"))
        .withColumn("FLT_DEP_1", f.col("FLT_DEP_1").cast("int"))
        .withColumn("FLT_DEP_IFR_2", f.col("FLT_DEP_IFR_2").cast("int"))
        .withColumn("DLY_ATC_PRE_2", f.col("DLY_ATC_PRE_2").cast("double"))
        .withColumn("FLT_DEP_3", f.col("FLT_DEP_3").cast("int"))
        .withColumn("DLY_ATC_PRE_3", f.col("DLY_ATC_PRE_3").cast("double"))
    )


for year in range(2016, 2025):
    url = f"{base_url}atc_pre_departure_delays_{year}.csv"
    print(f"Loading atc_pre_departure_delays {year}")
    df = load_data_from_url_as_spark_df(url, sep=",", quotechar='"')
    df = normalize_atc_pre_dep_schema(df)

    (
        df
        .transform(lambda x: add_metadata_columns(x, year, "atc_pre_departure_delays"))
        .writeTo("airport_delays.raw.atc_pre_dep")
        .option("mergeSchema", "true")
        .append()
    )


In [0]:
delta_table = DeltaTable.forName(spark, "airport_delays.raw.atc_pre_dep")
df = delta_table.toDF()
df.printSchema()
df.show(5, truncate=False)
print(f"Total rows: {df.count()}")

The en-route ATFM delay provides an indication of ATFM delays on the ground due to constraints en-route. <br>

In Europe, when traffic demand is anticipated to exceed the available capacity in en route centres or at airports, Air Traffic Control (ATC ) units may request the local Flow Management Position (FMP ) to instigate an Air Traffic Flow Management (ATFM) measure, or regulation . Aircraft expected to arrive during a period of congestion are given ATFM delay at their departure airport, under the authority of the Network Manager, in order to regulate the flow of traffic into the constrained downstream en route sector or airport, thus ensuring safety. <br>

The calculation of En-route ATFM delay is based on a well established and commonly accepted algorithm and has been in use as a commonly agreed proxy for en-route capacity shortfalls since 1999. <br>

ATFM delay computation <br>
The ATFM delays are calculated as the difference between the estimated take-off time calculated from the filed flight plan including updates and the calculated take-off time allocated by the central unit of ATFM. <br>

The reason for the regulation is indicated by the responsible Flow Management Position (FMP). <br>

The delay is attributed to the most constraining ATC unit. <br>

Please note that the delays caused by READY to Depart (REA) messages and ATFM slot extensions were included in ATFM delays until NM release 20.0. As of 4 April 2016, delays due to REA messages and slot extensions are not included any longer which reduces the amount of computed ATFM delay. <br>

Computation of traffic <br>
The number of flights is based on ANSP boundaries (which are not necessarily coincidental with the FIR or geographical boundaries of the State). <br>

| Column name     | Data source      | Label                  | Reason Group         | Column description                                             | Example   |
|-----------------|------------------|------------------------|----------------------|----------------------------------------------------------------|-----------|
| YEAR            | Network Manager  | YEAR                   |                      | Reference year                                                 | 2015      |
| MONTH_NUM       | Network Manager  | MONTH                  |                      | Month (numeric)                                                | 3         |
| MONTH_MON       | Network Manager  | MONTH_MON              |                      | Month (3-letter code)                                          | MAR       |
| FLT_DATE        | Network Manager  | FLT_DATE               |                      | Date of flight                                                | 17/03/2015|
| ENTITY_NAME     | PRU              | ENTITY_NAME            |                      | Entity name                                                   | FAB CE    |
| ENTITY_TYPE     | PRU              | ENTITY_TYPE            |                      | Type of the entity to which the data relates (ANSP, FAB, AREA) | FAB (AUA) |
| FLT_ERT_1       | Network Manager  | Flights                |                      | Total number of flights within the respective airspace       | 3853      |
| DLY_ERT_1       | Network Manager  | En-route ATFM delay    |                      | Minutes of en-route ATFM delay                                | 0         |
| DLY_ERT_A_1     | Network Manager  | A - Accident/Incident  | ER Disruptions       | Minutes of en-route ATFM delay with delay code A - Accident/Incident | 0     |
| DLY_ERT_C_1     | Network Manager  | C - ATC Capacity       | ER Capacity (ATC)    | Minutes of en-route ATFM delay with delay code C - ATC Capacity | 0       |
| DLY_ERT_D_1     | Network Manager  | D - De-icing           | ER Weather           | Minutes of en-route ATFM delay with delay code D - De-icing  | 0         |
| DLY_ERT_E_1     | Network Manager  | E - Equipment (non-ATC)| ER Disruptions       | Minutes of en-route ATFM delay with delay code E - Equipment (non-ATC) | 0     |
| DLY_ERT_G_1     | Network Manager  | G - Aerodrome Capacity | ER Capacity          | Minutes of en-route ATFM delay with delay code G - Aerodrome Capacity | 0     |
| DLY_ERT_I_1     | Network Manager  | I - Industrial Action (ATC) | ER Disruptions (ATC)| Minutes of en-route ATFM delay with delay code I - Industrial Action (ATC) | 0   |
| DLY_ERT_M_1     | Network Manager  | M - Airspace Management| ER Capacity          | Minutes of en-route ATFM delay with delay code M - Airspace Management | 0     |
| DLY_ERT_N_1     | Network Manager  | N - Industrial Action (non-ATC) | ER Disruptions | Minutes of en-route ATFM delay with delay code N - Industrial Action (non-ATC) | 0  |
| DLY_ERT_O_1     | Network Manager  | O - Other              | ER Disruptions       | Minutes of en-route ATFM delay with delay code O - Other     | 0         |
| DLY_ERT_P_1     | Network Manager  | P - Special Event      | ER Events            | Minutes of en-route ATFM delay with delay code P - Special Event | 0       |
| DLY_ERT_R_1     | Network Manager  | R - ATC Routeing       | ER Capacity          | Minutes of en-route ATFM delay with delay code R - ATC Routeing | 0        |
| DLY_ERT_S_1     | Network Manager  | S - ATC Staffing       | ER Staffing (ATC)    | Minutes of en-route ATFM delay with delay code S - ATC Staffing | 0        |
| DLY_ERT_T_1     | Network Manager  | T - Equipment (ATC)    | ER Disruptions (ATC) | Minutes of en-route ATFM delay with delay code T - Equipment (ATC) | 0       |
| DLY_ERT_V_1     | Network Manager  | V - Environmental Issues | ER Capacity        | Minutes of en-route ATFM delay with delay code V - Environmental Issues | 0    |
| DLY_ERT_W_1     | Network Manager  | W - Weather            | ER Weather           | Minutes of en-route ATFM delay with delay code W - Weather   | 0         |
| DLY_ERT_NA_1    | Network Manager  | NA - Not specified     | ER Disruptions       | Minutes of en-route ATFM delay with delay code NA - Not specified | 0       |
| FLT_ERT_1_DLY   | Network Manager  | FLT_ERT_1_DLY          |                      | Number of en-route ATFM delayed flights                      | 0         |
| FLT_ERT_1_DLY_15| Network Manager  | FLT_ERT_1_DLY_15       |                      | Number of en-route ATFM delayed flights (>15 min.)           | 0         |


Data is aggregated from 2008 to 2024 year in table **enroute_dly** <br>
Data link -> https://ansperformance.eu/reference/dataset/en-route-atfm-delay-aua/

In [0]:
# spark.sql("DROP TABLE IF EXISTS airport_delays.raw.enroute_dly")

In [0]:
with open(contracts_path + "/ert_dly_schema.json", "r") as fin:
    ert_dly_schema = StructType.fromJson(json.loads(fin.read()))

raw_ert_dly_schema = update_schema_with_metadata_fields(ert_dly_schema)

In [0]:
ert_dly_delta = DeltaTable.createIfNotExists(spark) \
    .tableName("airport_delays.raw.enroute_dly") \
    .addColumns(raw_ert_dly_schema) \
    .execute()


In [0]:
base_url = "https://www.eurocontrol.int/performance/data/download/csv/"

In [0]:
def normalize_ert_dly_schema(df):
    df = df.withColumn("YEAR", f.col("YEAR").cast("int")) \
           .withColumn("MONTH_NUM", f.col("MONTH_NUM").cast("int")) \
           .withColumn("FLT_ERT_1", f.col("FLT_ERT_1").cast("int")) \
           .withColumn("DLY_ERT_1", f.col("DLY_ERT_1").cast("double")) \
           .withColumn("DLY_ERT_A_1", f.col("DLY_ERT_A_1").cast("double")) \
           .withColumn("DLY_ERT_C_1", f.col("DLY_ERT_C_1").cast("double")) \
           .withColumn("DLY_ERT_D_1", f.col("DLY_ERT_D_1").cast("double")) \
           .withColumn("DLY_ERT_E_1", f.col("DLY_ERT_E_1").cast("double")) \
           .withColumn("DLY_ERT_G_1", f.col("DLY_ERT_G_1").cast("double")) \
           .withColumn("DLY_ERT_I_1", f.col("DLY_ERT_I_1").cast("double")) \
           .withColumn("DLY_ERT_M_1", f.col("DLY_ERT_M_1").cast("double")) \
           .withColumn("DLY_ERT_N_1", f.col("DLY_ERT_N_1").cast("double")) \
           .withColumn("DLY_ERT_O_1", f.col("DLY_ERT_O_1").cast("double")) \
           .withColumn("DLY_ERT_P_1", f.col("DLY_ERT_P_1").cast("double")) \
           .withColumn("DLY_ERT_R_1", f.col("DLY_ERT_R_1").cast("double")) \
           .withColumn("DLY_ERT_S_1", f.col("DLY_ERT_S_1").cast("double")) \
           .withColumn("DLY_ERT_T_1", f.col("DLY_ERT_T_1").cast("double")) \
           .withColumn("DLY_ERT_V_1", f.col("DLY_ERT_V_1").cast("double")) \
           .withColumn("DLY_ERT_W_1", f.col("DLY_ERT_W_1").cast("double")) \
           .withColumn("DLY_ERT_NA_1", f.col("DLY_ERT_NA_1").cast("double")) \
           .withColumn("FLT_ERT_1_DLY", f.col("FLT_ERT_1_DLY").cast("int")) \
           .withColumn("FLT_ERT_1_DLY_15", f.col("FLT_ERT_1_DLY_15").cast("int"))
    return df

for year in range(2008, 2025):
    url = f"{base_url}ert_dly_ansp_{year}.csv.bz2"
    print(f"Loading ert_dly_ansp {year}")

    df = load_data_from_url_as_spark_df(
        url,
        sep=",",
        quotechar='"',
        compression="bz2"
    )

    df = normalize_ert_dly_schema(df)

    (
        df
        .transform(lambda x: add_metadata_columns(x, year, "ert_dly_ansp"))
        .writeTo("airport_delays.raw.enroute_dly")
        .option("mergeSchema", "true")
        .append()
    )


In [0]:
delta_table = DeltaTable.forName(spark, "airport_delays.raw.enroute_dly")
df = delta_table.toDF()
df.printSchema()
df.show(5, truncate=False)
print(f"Total rows: {df.count()}")